In [1]:
using StatisticalRethinking
using CmdStan, StanMCMCChain

ProjDir = rel_path("..", "scripts", "12")
cd(ProjDir)

d = CSV.read(rel_path( "..", "data",  "Kline.csv"), delim=';');
size(d) # Should be 10x5

(10, 5)

New col log_pop, set log() for population data

In [2]:
d[:log_pop] = map((x) -> log(x), d[:population]);
d[:society] = 1:10;
first(d[[:culture, :population, :log_pop, :society]], 5)

m12_6 = "
data {
    int N;
    int N_societies;
    int total_tools[N];
    real logpop[N];
    int society[N];
}
parameters{
    real a;
    real bp;
    vector[N_societies] a_society;
    real<lower=0> sigma_society;
}
model{
    vector[N_societies] mu;
    sigma_society ~ cauchy( 0 , 1 );
    a_society ~ normal( 0 , sigma_society );
    bp ~ normal( 0 , 1 );
    a ~ normal( 0 , 10 );
    for ( i in 1:N ) {
        mu[i] = a + a_society[society[i]] + bp * logpop[i];
        mu[i] = exp(mu[i]);
    }
    total_tools ~ poisson( mu );
}
";

Define the Stanmodel and set the output format to :mcmcchain.

In [3]:
stanmodel = Stanmodel(name="m12.6",  model=m12_6, output_format=:mcmcchain);

Input data for cmdstan

In [4]:
m12_6_data = Dict("N" => size(d, 1),"N_societies" => 10,
"total_tools" => d[:total_tools], "logpop" => d[:log_pop],
"society" => d[:society]);

Sample using cmdstan

In [5]:
rc, chn, cnames = stan(stanmodel, m12_6_data, ProjDir, diagnostics=false, summary=false, CmdStanDir=CMDSTAN_HOME);

Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
Exception: normal_lpdf: Scale parameter is 0, but must be > 0!  (in '/Users/rob/.julia/dev/StatisticalRethinking/scripts/12/tmp/m12.6.stan' at line 17)

If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.



Describe the draws

In [6]:
describe(chn)

Iterations = 1:1000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 1000

Empirical Posterior Estimates:
                   Mean           SD         Naive SE        MCSE         ESS    
         lp__  927.843931750  3.131010809 0.049505627677 0.1065060463  864.212627
accept_stat__    0.936031801  0.086342025 0.001365187289 0.0017614645 1000.000000
   stepsize__    0.053839025  0.006320427 0.000099934727 0.0010119518   39.009752
  treedepth__    5.801000000  0.486783565 0.007696723971 0.0218174242  497.811428
 n_leapfrog__   68.334000000 30.240917186 0.478150884206 1.1788264917  658.097352
  divergent__    0.000000000  0.000000000 0.000000000000 0.0000000000         NaN
     energy__ -921.339582500  4.028494575 0.063696091996 0.1317069550  935.552466
            a    1.105386906  0.767573265 0.012136398937 0.0231058873 1000.000000
           bp    0.260627280  0.082810742 0.001309352804 0.0024350570 1000.000000
  a_society.1   -0.200100687  0.250850004 0.003966286816 0.00707

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*